In [23]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

## 1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances? 

Roughly $log(l)+1 = log(\frac{1,000,000 + 1}{2}) + 1 = log(1000000+1) - log(2) + 1 = log(1000000 + 1) \approx 20$


## 2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater than or always lower/greater? 
A node is generally lower Gini impurity value than its parent. This is because the cost function in the CART algorithm seeks to minimize the Gini impurity metric of both children but it does so as a combination of left and right nodes: $\frac{m_{left}}{m}Gini_{left} + \frac{m_{right}}{m}Gini_{right}$. So a single node could be higher than its parent but that score will be compensated by a much lower score by the other child node. 

## 3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth? 
Yes, this minimize how much the algorithm fits the data for splitting. 

## 4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features? 
A decision tree does not require feature scaling, so doing so will not help underfitting. To help underfitting, try increaseing the max depth of the tree or loosening restrictions on parameters such as min leaf size. 

## 5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances? 
1 hour = $n*1mlog(1m) \implies n*10mlog(10m) = n*10m(log(10) + log(m)) = n*10m*log(10) + 10*n*1mlog(m) \approx 1.66*1\; hour + 10\;hours$, by taking the ratio of $n*10m*log(10) \text{ to } n*1m*log(1m)$ 

## 6. If your training set contains 100,000 instances, will setting presort=True speed up training? 
This seems to be beyond what would help with training speed up, which is limited to sorting when instances are a few thousand. Maybe it depends on the machine and size of the training set at 100k. 


## 7. Train and fine tune a Decision Tree on the moon dataset

In [13]:
X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [38]:
pline = Pipeline([("dtree", DecisionTreeClassifier())])
grid = [{"dtree__max_depth": [6, 7, 8, 9, 10, 11, 12],
         "dtree__criterion": ["gini", "entropy"],
         "dtree__splitter": ['best', 'random'],
         "dtree__max_leaf_nodes": [14, 15, 16, 17, 18, 19, 20]
        }]

grid_cv = GridSearchCV(pline, grid, cv=5, n_jobs=4, verbose=2)

In [39]:
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 196 candidates, totalling 980 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 136 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 980 out of 980 | elapsed:    3.5s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('dtree', DecisionTreeClassifier())]),
             n_jobs=4,
             param_grid=[{'dtree__criterion': ['gini', 'entropy'],
                          'dtree__max_depth': [6, 7, 8, 9, 10, 11, 12],
                          'dtree__max_leaf_nodes': [14, 15, 16, 17, 18, 19, 20],
                          'dtree__splitter': ['best', 'random']}],
             verbose=2)

In [40]:
grid_cv.best_score_

0.85375

In [41]:
y_hat = grid_cv.best_estimator_.predict(X_test)

In [42]:
accuracy_score(y_test, y_hat)

0.8575

In [43]:
grid_cv.best_params_

{'dtree__criterion': 'gini',
 'dtree__max_depth': 7,
 'dtree__max_leaf_nodes': 17,
 'dtree__splitter': 'best'}

# Chapter 7 

## 1. If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why? 
The ensemble results can be better than each predictor, so long as each predictor is independent - very different types are best. Also better if each model is trained on different instances (e.g. bagging/pasting).

## 2. What is the difference between hard and soft voting classifier? 
Hard voting classifier chooses the majority class and soft voting chooses class with highest average probability, which then gives higher weight to more confident classifers where hard voting does not. Soft voting does require each model to compute class probabilities

## 3. Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, Random Forests, or stacking ensembles? 
Yes, it is possible to speed up training of a bagging ensemble via distributing it across servers. This could be achieved by only transferring the subset of data to the appropriate node. This is also the case with pasting, but since this is done without replacement, care would need to be take so that data is not sent more than once. Boosting ensembles cannot be sped up in this way due to the need for sequential processing of each predictor. Random Forests should be able to ran distributively. Stacking also would be able to done in a distributed fashiong but there will need to be a process of forking and joining to create the necessary training data sets. 

## 4. What is the benefit of out-of-bag evaluation? 
It is a means to easily evaluate the performance on how well the model will perform on fresh training instances. And this is easily available since the process of bagging/pasting is that certain instances will have not been chosen and can be used for evaluation. Additionally this frees up more instances for training as a special validation set is not needed.

## 5. What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster than regular Random Forests? 
Decision Trees searches for the best feature/threshold combination for purity to split a node on. Random Forests split based upon the best feature/threshold combination based upon a random subset of features. Extra-Trees do the same thing, however it splits also on a random threshold and not the best. This helps in regularization and is good if a RandomForest is overfitting. Also since it does not look for an optimized set of features to thresholds, training is faster. 

## 6. If your AdaBoost ensemble underfits the training data, which hyperparameters should you tweak and how? 
You can increase the number of the number predictors in the chain or reduce the regularization of each component model. 

## 7. If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate? 
You would reduce the learning rate, which decreases the amount each individual model contributes to the ensemble. This requires more component models but extends better. One can also try early stopping to see if you are using too many predictors. 

In [1]:
# 8. MNIST data: training 50k, 10k validation, & test 10k; traing RF, EX Tree, SVM classifiers; Combine as ensemble that outperforms each indiv using 
# soft or hard voting;  

In [ ]:
# 9. Create Stacking ensemble with 1 blender 